In [1]:
year = 2022
month = 'Febrero'
week = 9

# Libraries

In [2]:
import pandas as pd
import pyodbc
from datetime import datetime
import csv

# Parameters

In [3]:
filename = '{0}_{1}_Nadro.xlsx'.format(year, str(week).zfill(2))

path_export = r'\\NASPRO.infovisiontv.com\Respaldo_Operacion_Cargas\Nadro\{0}\Layout Diario'
path_export = path_export.format(year)

filename_export = '{0}_{1}_NadroV5GLI_{3}-{2}-{0}_S{1}_Real test.txt'

path_backup = r'\\NASPRO.infovisiontv.com\Respaldo_Operacion_Cargas\Nadro\{0}'
path_backup = path_backup.format(year)

path_load = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\MEX\Carga\Semanales\Nadro\Data\{0}\2. {2}\S {1}'
path_load = path_load.format(year, str(week).zfill(2), month)

path_file_catalog = r'\\NASPRO.infovisiontv.com\Respaldo_Operacion_Cargas\Nadro\Catalogos.xlsx'
sheet_sucs = 'Sucursales'
sheet_inv = 'Inventarios'

In [4]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=SFEDWH01;'
                      'Trusted_Connection=yes;')

In [5]:
qdates = '''SELECT SemAnio,
                  SemNumero,
                  SemInicio,
                  SemFin
                  FROM Gnm_MasterOp.dbo.CatSemanas
                  WHERE SemAnio>={0} AND SemNumero BETWEEN {1} AND {2}'''

qdates = qdates.format(year, week - 1, week)

# Load Data

In [6]:
ddates = pd.read_sql(qdates, conn)

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
date = ddates['SemFin'][ddates['SemNumero'] == ddates['SemNumero'].max()]
day = date.squeeze().day
month = date.squeeze().month

Cargamos 2 catálogos:
- *sucs* es un catálogo referente a las sucursales.
- *inv* es un catálogo sobre el inventario.

In [8]:
sucs = pd.read_excel(path_file_catalog, sheet_name=sheet_sucs, usecols='A, D')
inv = pd.read_excel(path_file_catalog, sheet_name=sheet_inv)

Filtramos solo las columnas que nos interesan

In [9]:
cols = ['EAN', 'SAP', 'MATERIAL', 'CS', 'PFCIA', 'PPUB', 'CTRO', 'EXIS', 'TRANS', 'TOT PZAS']
data = pd.read_excel(path_load + '\\' + filename)
data = data[cols]

# Data Cleaning

En ocasiones, el reporte tiene filas en blanco al final, por lo que se las quitamos en seguida:

In [10]:
index_to_remove = data[-10:].T.isnull().sum()[data[-10:].T.isnull().sum()==9].index
data = data.iloc[[i for i in data.index if i not in index_to_remove], :]
data.reset_index(drop=True, inplace=True)

In [11]:
# Creación de columna "DEMANDA"
data['DEMANDA'] = data['TOT PZAS']*4
# Limpieza de columna "EAN"
data['EAN'] = data['EAN'].map(int)
data.loc[data['EAN'].isin([7501004227620, 7501004227644, 7501004226890]), 'DEMANDA'] = data.loc[data['EAN'].isin([7501004227620, 7501004227644, 7501004226890]), 'DEMANDA']*3
data.loc[data['EAN']==7501004229983, 'EAN'] = 650240010736
# Limpieza de columna "SAP"
data['SAP'] = data['SAP'].map(int)
data = data[~data['SAP'].isin([44362, 44363])]
# Imputamos 0 a la columna "TRANS"
data['TRANS'].fillna(0, inplace=True)
# Cambiamos el tipo de dato a entero a la columna "CTRO"
data['CTRO'] = data['CTRO'].map(int)

In [12]:
# Renombramos algunas columnas de los datos semanales
data.rename({
    'SAP':'CODIGO', 
    'MATERIAL':'DESCRIPCION',
    'CTRO':'Sucursal ID',
    'EXIS':'EXISTENCIA', 
    'TRANS':'TRANSITO'
}, axis=1, inplace=True)

In [13]:
# Renombramos la columna Concat-Suc de sucs
sucs.rename({
    'Concat-Suc':'SUCURSAL'
}, axis=1, inplace=True)

# Merge

In [14]:
# Cruce con sucs
rows_before = data.shape[0]
data = data.merge(sucs, on='Sucursal ID', how='left')
rows_after = data.shape[0]
print(rows_before == rows_after)

True


In [15]:
# Llave para cruzar con inv
data['Llave'] = data['CODIGO'].map(str) + '-' + data['SUCURSAL']

In [16]:
# Cruce con inv
rows_before = data.shape[0]
data = data.merge(inv, on='Llave', how='left')
rows_after = data.shape[0]
print(rows_before == rows_after)

True


## Cleaning

Aquí ya con las columnas de los catálogos se limpian los datos antes de ser exportados:

In [17]:
data.loc[data['SUCURSAL']=='50CEDI', 'TRANSITO'] = 0

In [18]:
# Imputación de datos con 0
for c in ['DPZTL_NOV_18', 'DPZTL_DIC_18', 'DPZTL_ENE_19']:
    data[c].fillna(0, inplace=True)
# Cambia el tipo de dato a entero
for c in ['DPZTL_DIC_18', 'DPZTL_ENE_19', 'DPZTL_NOV_18', 'EXISTENCIA', 'TRANSITO']:
    final[c] = final[c].map(int)

In [19]:
# Nos quedamos con registros únicos
final = data.pivot_table(index=['EAN', 
                                'CODIGO', 
                                'DESCRIPCION',
                                'CS',
                                'PFCIA', 
                                'PPUB',
                                'SUCURSAL'], 
                        values= ['EXISTENCIA',
                                 'TRANSITO', 
                                 'DEMANDA',
                                 'DPZTL_NOV_18',
                                 'DPZTL_DIC_18',
                                 'DPZTL_ENE_19'], 
                        aggfunc='sum').reset_index()

In [20]:
# Se crea una columna nueva "DOIS"
final['DOIS'] = 0

# Export

Creo una lista *cols* que tiene el orden de columnas de cómo se exportará el archivo

In [36]:
cols = [
    'EAN',
    'CODIGO',
    'DESCRIPCION',
    'CS',
    'PFCIA',
    'PPUB',
    'SUCURSAL',
    'EXISTENCIA',
    'TRANSITO',
    'DEMANDA',
    'DPZTL_NOV_18',
    'DPZTL_DIC_18',
    'DPZTL_ENE_19',
    'DOIS'
    ]

In [37]:
filename_export = filename_export.format(year, week, month, day)
# csv.QUOTE_NONNUMERIC le agrega '"' a las columnas object
final[cols].to_csv(path_backup + '\\' + filename_export, index=False, quoting=csv.QUOTE_NONNUMERIC, sep='\t')